In [1]:
import numpy as np    
import pandas as pd    
import matplotlib.pyplot as plt  
import os  
import cartopy.crs as ccrs  
import cartopy.feature as cfeature  
import matplotlib.ticker as mticker  
from matplotlib.ticker import MultipleLocator, FuncFormatter
import matplotlib.colors as mcolors  
import gsw

In [2]:
df_0 = pd.read_excel(r'D:\ze_1112\最新数据\NORC2022-09_Chl-a+营养盐_20240422更新_绘图使用数据.xls')
# df.dropna(inplace=True)
df_0 = df_0.rename(columns={'站位':'站位名'})
df_0

,站位名,水深（m）,叶绿素浓度（μg/L）,NH4+-N (μmol/L),NO3-N (μmol/L),NO2-N (μmol/L),PO4-P (μmol/L),SiO4-Si (μmol/L)
0,QB08,5,0.032143,0.03,0.0703,0.0267,0.1073,1.2367
1,QB08,75,0.068734,未检出,0.1103,0.0067,0.1083,1.2567
2,QB08,150,NaN,未检出,0.7503,0.0567,0.1643,1.5367
3,QB08,700,NaN,未检出,35.48333,0.02,2.31,72.4100
4,QB08,1000,NaN,未检出,38.73333,0.02,2.51,104.7600
...,...,...,...,...,...,...,...,...
493,N21-10,1000,NaN,未检出,38.24333,0.02,2.58,114.5600
494,N21-10,2000,NaN,未检出,38.24333,0.02,2.51,139.5000
495,N21-10,3000,NaN,未检出,37.68333,0.02,2.5,140.1100
496,N21-10,4000,NaN,未检出,36.28333,0.03,2.32,137.5900


In [3]:
stations = []
for i in df_0['站位名']:
    stations.append(i)
stations = list(set(stations))
print(stations)

['N21-10', 'E130-12', 'E163-06', 'E163-04', 'E130-04', 'E130-08', 'E130-14', 'E163-05', 'N03-04', 'N03-02', 'E163-02', 'E140-11', 'E163-07', 'E140-07', 'QB06', 'FB01', 'E163-03', 'N21-13', 'E130-13', 'N21-12', 'E130-02', 'N21-14', 'E163-04s', 'N03-03', 'E130-03', 'QB08', 'E140-03orQB01', 'E130-09', 'QB03', 'E140-08', 'E130-10', 'QB07', 'E163-06s', 'QB04', 'E130-07', 'E130-11', 'E130-06', 'N21-11', 'E163-01', 'E130-05']


In [4]:
def dms_to_decimal(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [5]:
stations_ctd = os.listdir(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表')
df_1 = pd.DataFrame()
 
for i in stations_ctd:
    if i.endswith('.xls'):
        df = pd.read_excel(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表\%s' % i, skiprows=3, usecols=[1,5,6,8,10,11,13,16], names=['站位名', '纬度', '经度', '水深（m）', '压力', '水温', '盐度', '溶解氧'])
        df = df.drop(0)
        name = df['站位名'].values
        depth = df['水深（m）'].values
        preasure = df['压力'].values
        o2 = df['溶解氧'].values.astype(float)
        salinity = df['盐度'].values.astype(float)
        t = df['水温'].values.astype(float)
        p = df['压力'].values.astype(float) * 0.1 -10.1325
        lon = df['经度'].apply(dms_to_decimal).values.astype(float)
        lat = df['纬度'].apply(dms_to_decimal).values.astype(float)
        # lonlat.append((longitude[3], latitude[3], name[3]))
        df_1 = pd.concat([df,df_1])
df_21 = pd.merge(df_1,df_0, on=['站位名', '水深（m）'])       
# lonlat = pd.DataFrame(lonlat, columns=['lon', 'lat', '站位名'])
# data_path = pd.merge( lonlat, df_0, on='站位名')
df_21 = df_21.dropna(subset=['SiO4-Si  (μmol/L)'])


In [94]:
name = df_21['站位名'].values
name = list(set(name))
name.sort()
Philippine_Basin = [n for n in name if '130' in n or '21' in n]
West_Caroline_Basin = [n for n in name if '140' not in n and 'FB01' in n]
Central_Pacific_Basin = [n for n in name if '163' in n]
East_Caroline_Basin = [n for n in name if 'N03' in n and 'QB03' not in n]
West_Mariana_Basin = [n for n in name if '165' in n or 'QB06' in n or 'QB07' in n]
plt.rcParams['font.size'] = 18

all_colors = all_colors = [
    '#FF5733', '#33FF57', '#3357FF', '#F333FF', '#33FFF5', '#3CB371', '#FF3357', '#5733FF',
    '#33F5FF', '#FF33F5', '#F533FF', '#33FF33', '#FF3333', '#3333FF', '#3CB371', '#33FFFF',
    '#FF33FF', '#33FF33', '#FF3333', '#3333FF', '#3CB371', '#33FFFF', '#FF33FF', '#33FF33',
    '#FF3333', '#3333FF', '#3CB371', '#33FFFF', '#FF33FF', '#33FF33', '#FF3333', '#3333FF',
    '#3CB371', '#33FFFF', '#FF33FF', '#33FF33', '#FF3333', '#3333FF', '#3333FF', '#FF5733'
]

all_markers = [
    'o', 's', 'D', '^', 'v', 'p', 'h', '8', 'X', '*',  # 圆圈, 正方形, 菱形, 三角形(向上), 三角形(向下), 五角星, 六边形, 八角形, 叉号, 星号
    'o', 's', 'D', '^', 'v', 'p', 'h', '8', 'X', '*',  
    'o', 's', 'D', '^', 'v', 'p', 'h', '8', 'X', '*',  
    'o', 's', 'D', '^', 'v', 'p', 'h', '8', 'X', '*',  
    'o', 's', 'D', '^', 'v', 'p', 'h', '8', 'X', '*', '*'   
]


fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)

for indx, station in enumerate(name):
    df_2 = df_21[df_21['站位名']==station]
    depth = df_2['水深（m）'].values
    preasure = df_2['压力'].values
    o2 = df_2['溶解氧'].values.astype(float)
    salinity = df_2['盐度'].values.astype(float)
    t = df_2['水温'].values.astype(float)
    p = df_2['压力'].values.astype(float) * 0.1 -10.1325
    lon = df_2['经度'].apply(dms_to_decimal).values.astype(float)
    lat = df_2['纬度'].apply(dms_to_decimal).values.astype(float)
    sio4 = df_2['SiO4-Si  (μmol/L)'].values

    AT = gsw.SA_from_SP(salinity, p, lon, lat)
    PT = gsw.pt0_from_t(AT, t, p)
    CT = gsw.CT_from_pt(AT, PT)
    ax1.scatter(PT, sio4, s=70, color=all_colors[indx], marker=all_markers[indx], label=station, alpha=0.7)
ax1.spines[:].set_linewidth(2)
ax1.set_xlabel('Potential Temperature (°C)')
ax1.set_ylabel('SiO4-Si (μmol/L)')
ax1.grid(linestyle='--', alpha=0.5)

plt.legend(fontsize=11,ncols=2)
# plt.show()
plt.savefig('PT_sio4.png', dpi=300, bbox_inches='tight')
plt.close()

In [69]:
name = df_21['站位名'].values
name = list(set(name))
name.sort()
Philippine_Basin = [n for n in name if '130' in n or '21' in n]
West_Caroline_Basin = [n for n in name if '140' not in n and 'FB01' in n]
Central_Pacific_Basin = [n for n in name if '163' in n]
East_Caroline_Basin = [n for n in name if 'N03' in n and 'QB03' not in n]
West_Mariana_Basin = [n for n in name if '165' in n or 'QB06' in n or 'QB07' in n]




In [95]:
import matplotlib.pyplot as plt
import gsw

# 获取所有站位的名称并去重
name = df_21['站位名'].values
name = list(set(name))
name.sort()

# 定义不同海区的站位名列表
Philippine_Basin = [n for n in name if '130' in n or '21' in n]
West_Caroline_Basin = [n for n in name if '140' not in n and 'FB01' in n]
Central_Pacific_Basin = [n for n in name if '163' in n]
East_Caroline_Basin = [n for n in name if 'N03' in n and 'QB03' not in n]
West_Mariana_Basin = [n for n in name if '165' in n or 'QB06' in n or 'QB07' in n]

# 定义颜色和标记列表
# colors = ['#8B8970' #8B8970 '#3CB371', '#8B2500', '#E066FF']
colors = ['#E066FF','#8B2500','#3CB371','#33FF57','#8B8970']
markers = ['o', 's', 'D', '^', 'v']

# 设置字体大小
plt.rcParams['font.size'] = 18

# 创建图形和子图
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(111)

# 定义海区列表
basins = [West_Mariana_Basin, West_Caroline_Basin, Philippine_Basin, East_Caroline_Basin, Central_Pacific_Basin ]
basin_names = ['West_Mariana_Basin', 'West_Caroline_Basin', 'Philippine_Basin', 'East_Caroline_Basin', 'Central_Pacific_Basin']

# 遍历每个海区
for basin_idx, basin in enumerate(basins):
    for indx, station in enumerate(basin):
        df_2 = df_21[df_21['站位名'] == station]
        depth = df_2['水深（m）'].values
        pressure = df_2['压力'].values
        o2 = df_2['溶解氧'].values.astype(float)
        salinity = df_2['盐度'].values.astype(float)
        t = df_2['水温'].values.astype(float)
        p = df_2['压力'].values.astype(float) * 0.1 - 10.1325
        lon = df_2['经度'].apply(dms_to_decimal).values.astype(float)
        lat = df_2['纬度'].apply(dms_to_decimal).values.astype(float)
        sio4 = df_2['SiO4-Si  (μmol/L)'].values

        AT = gsw.SA_from_SP(salinity, p, lon, lat)
        PT = gsw.pt0_from_t(AT, t, p)
        CT = gsw.CT_from_pt(AT, PT)
        ax1.scatter(PT, sio4, s=70, color=colors[basin_idx], marker=markers[basin_idx], label=basin_names[basin_idx], alpha=0.8)

# 设置边框线宽
ax1.spines[:].set_linewidth(2)
ax1.set_xlabel('Potential Temperature (°C)')
ax1.set_ylabel('SiO4-Si (μmol/L)')
ax1.grid(linestyle='--', alpha=0.5)
# 添加图例
plt.legend(fontsize=11, ncols=2)

# 保存图像
plt.savefig('PT_sio4_haiqu.png', dpi=300, bbox_inches='tight')
plt.close()


In [9]:
import matplotlib.pyplot as plt
import gsw

# 获取所有站位的名称并去重
name = df_21['站位名'].values
name = list(set(name))
name.sort()

# 定义不同海区的站位名列表
Philippine_Basin = [n for n in name if '130' in n or '21' in n]
West_Caroline_Basin = [n for n in name if '140' not in n and 'FB01' in n]
Central_Pacific_Basin = [n for n in name if '163' in n]
East_Caroline_Basin = [n for n in name if 'N03' in n and 'QB03' not in n]
West_Mariana_Basin = [n for n in name if '165' in n or 'QB06' in n or 'QB07' in n]

# 定义颜色和标记列表
all_colors = ['#8B8970', '#FFA54F', '#3CB371', '#8B2500', '#E066FF']
# all_colors.reverse()
all_markers = ['o', 'D', 'X', '*', '^']
# 设置字体大小
plt.rcParams['font.size'] = 18

# 创建图形和子图
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(111)

# 定义海区列表
basins = [West_Mariana_Basin, West_Caroline_Basin, Philippine_Basin, East_Caroline_Basin, Central_Pacific_Basin ]
basin_names = ['Central Pacific Basin','East Caroline Basin','Philippine Basin','West Caroline Basin','West Mariana Basin']

# 遍历每个海区
for basin_idx, basin in enumerate(basins):
    for indx, station in enumerate(basin):
        df_2 = df_21[df_21['站位名'] == station]
        depth = df_2['水深（m）'].values
        pressure = df_2['压力'].values
        o2 = df_2['溶解氧'].values.astype(float)
        salinity = df_2['盐度'].values.astype(float)
        t = df_2['水温'].values.astype(float)
        p = df_2['压力'].values.astype(float) * 0.1 - 10.1325
        lon = df_2['经度'].apply(dms_to_decimal).values.astype(float)
        lat = df_2['纬度'].apply(dms_to_decimal).values.astype(float)
        sio4 = df_2['SiO4-Si  (μmol/L)'].values

        AT = gsw.SA_from_SP(salinity, p, lon, lat)
        PT = gsw.pt0_from_t(AT, t, p)
        CT = gsw.CT_from_pt(AT, PT)
        
        # 只在每个海区的第一个站位时添加图例标签
        if indx == 0:
            ax1.scatter(PT, sio4, s=70, color=all_colors[basin_idx], marker=all_markers[basin_idx], label=basin_names[basin_idx], alpha=0.8)
        else:
            ax1.scatter(PT, sio4, s=70, color=all_colors[basin_idx], marker=all_markers[basin_idx], alpha=0.8)

# 设置边框线宽
ax1.spines[:].set_linewidth(2)
ax1.set_xlabel('Potential Temperature (°C)')
ax1.set_ylabel('SiO4-Si (μmol/L)')
ax1.grid(linestyle='--', alpha=0.5)
# 添加图例
plt.legend(markerscale=2)

# 保存图像
plt.savefig('PT_sio4_haiqu.png', dpi=300, bbox_inches='tight')
plt.close()
